In [1]:
# Module Importations


In [ ]:
# Project Module Importations

In [1]:
# Pyspark Import and Setup
import findspark
findspark.init()
import pyspark

ValueError: Couldn't find Spark, make sure SPARK_HOME env is set or Spark is in an expected location (e.g. from homebrew installation).

In [ ]:
# Constants (i.e. for saving figures)